# GroupEngAssign

> Invoke Package Group Eng to Assign Students in Groups

In [ ]:
#| default_exp assign

In [ ]:
#| hide
from nbdev.showdoc import *
import os
import pandas as pd
import shutil

In [ ]:
#| export
import GroupEng
import canvasapi
import github
from CanvasGroupy import GitHubGroup, CanvasGroup

In [ ]:
#| export
class AssignGroup:
    def __init__(self,
                 ghg: GitHubGroup, # authenticated GitHub object
                 cg: CanvasGroup, # authenticated canvas object
                 groupeng_config="", # Directory for the GroupEng config yml file
                ):
        "Initializer for Assign Group"
        self.status = None
        self.out_dir = None
        self.prefix = None
        self.cg = cg
        self.ghg = ghg
        # Initialize if appropriate parameters are defined
        if groupeng_config != "":
            self.assign_groups(groupeng_config)

    def assign_groups(self,
                      groupeng_config:str, # Directory for the GroupEng config yml file
                      assign_canvas_group=False, # directly assign canvas groups
                      create_gh_repo=False, # directly create GitHub repos
                      username_quiz_id=-1, # username quiz id from canvas course
                      in_group_category="", # specify which group category the group belongs to
                      suffix="", # suffix to the group name
                     ) -> (bool, str): # Status and output directory of the compiled file.
        status, out_dir = GroupEng.run(groupeng_config)
        self.status, self.out_dir = status, out_dir
        file = os.path.split(groupeng_config)[1]
        self.prefix = os.path.splitext(file)[0]
        if assign_canvas_group:
            if self.cg.group_category is None and in_group_category == "":
                raise ValueError("Have to specify in_group_category to create canvas group")
            self.create_canvas_group(in_group_category, suffix)
        if create_gh_repo:
            if username_quiz_id == -1:
                raise ValueError("Have to specify the canvas username quiz id")
            self.create_github_group(username_quiz_id)
        return status, out_dir

    def create_canvas_group(self,
                            in_group_category="", # specify which group category the group belongs to
                            suffix="", # suffix to the group name
                            ):
        "Create canvas groups based on the generated group configuration"
        if self.out_dir is None:
            raise ValueError("The group configuration has not been set. Please assign group via assign_groups")
        if self.cg.group_category is None:
            raise ValueError("The group category has not been set.")
        if in_group_category == "":
            in_group_category = self.cg.group_category.name
        # load the generated configuration file
        groups_generated_fp = os.path.join(self.out_dir, f"{self.prefix}_groups.csv")
        with open(groups_generated_fp, "r") as f:
            groups = f.read().splitlines()
        # create canvas groups for each.
        for group in groups:
            group = group.replace(" ", "").split(",")
            group_name, group_members = group[0], group[1:]
            self.cg.assign_canvas_group(
                group_name=f"{group_name}{suffix}",
                group_members=group_members,
                in_group_category=in_group_category
            )

    def create_github_group(self,
                            username_quiz_id:int # username quiz id from canvas course
                            ):
        github_usernames = self.cg.fetch_username_from_quiz(username_quiz_id)
        self.cg.set_group_category(cg.group_category.name)
        groups = self.cg.group_to_emails
        repos = []
        for group_name, members in groups.items():
            group_git_usernames = []
            for email in members:
                try:
                    # try to get the git username for each student.
                    # not all students completed their quiz.
                    group_git_usernames.append(github_usernames[email])
                except KeyError:
                    print(f"{email}'s GitHub Username not found")
            repo = self.ghg.create_group_repo(
                repo_name=group_name,
                collaborators=group_git_usernames,
                permission="write",
                repo_template="COGS118A/group_template",
                rename_files={
                    "Checkpoint_groupXXX.ipynb": f"Checkpoint_{group_name}.ipynb",
                    "FinalProject_groupXXX.ipynb": f"FinalProject_{group_name}.ipynb",
                    "Proposal_groupXXX.ipynb": f"Proposal_{group_name}.ipynb"
                },
                private=True,
                description=f"COGS118A Final Project {group_name} Repository",
                team_slug="Instructors_Sp23",
                team_permission="admin"
            )
            print("")
            repos.append(repo)
        return repos



In [ ]:
# Create authenticated objects
ghg = GitHubGroup("../../../credentials.json",
                 "COGS118A"
                 )
cg = CanvasGroup("../../../credentials.json",
                 course_id=45532,
                 )
# create assign group object
ag = AssignGroup(ghg, cg)

Successfully Authenticated. GitHub account:  scott-yj-yang 
Target Organization Set:  COGS118A 
Authorization Successful!
Course Set:  COGS 195 - Instructional Apprenticeship - Fleischer [SP23] 
Getting List of Users... This might take a while...
Users Fetch Complete! The course has 5 students.


In [ ]:
# create a group category to hold students
cg.create_group_category({"name": "Project 1"})

GroupCategory(_requester=<canvasapi.requester.Requester object>, id=16456, name=Project 1, role=None, self_signup=None, group_limit=None, auto_leader=None, created_at=2023-05-17T20:38:56Z, created_at_date=2023-05-17 20:38:56+00:00, context_type=Course, course_id=45532, groups_count=0, unassigned_users_count=5, protected=False, allows_multiple_memberships=False, is_member=False)

In [ ]:
# assign, create both Canvas and GitHub Group in one call
status, out_dir = ag.assign_groups("../data/195_group_specification.groupeng",
                                   assign_canvas_group=True,
                                   create_gh_repo=True,
                                   username_quiz_id=139925,
                                   in_group_category="Project 1",
                                   suffix="-SP23-Testing"
                                   )

['H', 'B', '-']
['B', 'H', '-']
['-', 'H', 'B']
['B', 'H', '-']
['B', '-', 'H']
['-', 'B', 'H']
[None, 3.9, 3.1]
[3.9, 3.1, None]
[3.4, 2.5, 2.1]
[3.9, None, 3.1]
[3.4, 2.1, 2.5]
[3.4, 2.1, 2.5]
In Group Set: Project 1,
Group Group1-SP23-Testing Created!
Member dol005 Joined group Group1-SP23-Testing
Member xiw013 Joined group Group1-SP23-Testing
In Group Set: Project 1,
Group Group2-SP23-Testing Created!
Member jiz088 Joined group Group2-SP23-Testing
Member jiz100 Joined group Group2-SP23-Testing
Member nmackler Joined group Group2-SP23-Testing
Quiz: GitHub Username fetch! 
Generating Student Analaysis...
[====================] 100%
Report Generated!
The Question asked is 1399692: In plain text, what is your GitHub Username? Absolutely no typo, no extra space, no hyperlink please.. 
Make sure this is the correct question where you asked student for their GitHub id.
If you need to change the index of columns, change the col_index argument of this call.
dol005's GitHub Username not foun

The false means that at least one requirement is not satisfied. We can take a look at the file that was generated.

## API Doc

In [ ]:
show_doc(AssignGroup)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/assign.py#L13){target="_blank" style="float:right; font-size:smaller"}

### AssignGroup

>      AssignGroup (ghg:CanvasGroupy.github.GitHubGroup,
>                   cg:CanvasGroupy.canvas.CanvasGroup, groupeng_config='')

Initializer for Assign Group

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| ghg | GitHubGroup |  | authenticated GitHub object |
| cg | CanvasGroup |  | authenticated canvas object |
| groupeng_config | str |  | Directory for the GroupEng config yml file |

In [ ]:
show_doc(AssignGroup.assign_groups)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/assign.py#L29){target="_blank" style="float:right; font-size:smaller"}

### AssignGroup.assign_groups

>      AssignGroup.assign_groups (groupeng_config:str,
>                                 assign_canvas_group=False,
>                                 create_gh_repo=False, username_quiz_id=-1,
>                                 in_group_category='', suffix='')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| groupeng_config | str |  | Directory for the GroupEng config yml file |
| assign_canvas_group | bool | False | directly assign canvas groups |
| create_gh_repo | bool | False | directly create GitHub repos |
| username_quiz_id | int | -1 | username quiz id from canvas course |
| in_group_category | str |  | specify which group category the group belongs to |
| suffix | str |  | suffix to the group name |
| **Returns** | **(<class 'bool'>, <class 'str'>)** |  | **Status and output directory of the compiled file.** |

In [ ]:
show_doc(AssignGroup.create_canvas_group)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/assign.py#L51){target="_blank" style="float:right; font-size:smaller"}

### AssignGroup.create_canvas_group

>      AssignGroup.create_canvas_group (in_group_category='', suffix='')

Create canvas groups based on the generated group configuration

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| in_group_category | str |  | specify which group category the group belongs to |
| suffix | str |  | suffix to the group name |

In [ ]:
show_doc(AssignGroup.create_github_group)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/assign.py#L76){target="_blank" style="float:right; font-size:smaller"}

### AssignGroup.create_github_group

>      AssignGroup.create_github_group (username_quiz_id:int)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| username_quiz_id | int | username quiz id from canvas course |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()